In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pyLIA
import pysfmov as sfmov

In [15]:
E = 75*10**9# Pa Young Modulus
ni = 0.33 # Poisson ratio
filename = './data/rec44.sfmov'
data = sfmov.get_data(filename)
fs = 400
fl = 40
location = 56,38,30,70 # strain gauge area
eps = np.random.rand(1000,3)*1e-4

In [35]:
class calibration():    
    '''
    Thermoelastic coefficient f
    
    
    Reference:
    ---------
    Kobayashi, A. (1993). 
    Handbook of Experimental Mechanics, London: 2nd rev. Edition: Society for Experimental Mechanics.
    
    '''
        

    
    def get_strain(eps, configuration = None):
        
        '''
        Calculate strain for the calibration procedure from strain-gauge acquisition
        
        Input:
        -----
        eps : array_like
            Rows: Time series of measurement values
            Columns: Each measured strain-gague
                
        configuration : {'90', '120'}, optional
            In case of three strain-gauges usage, define the geometry configuration of the rosette.
            Default to None.
        
        Return:
        ------
        strain : float
            Strain to used in the calibration procedure
        
        '''
        
        if eps.shape[1] == 1:
            return np.max(eps)
        
        elif eps.shape[1] == 2:
            return (np.max(eps[:,0]) + np.max(eps[:,1])) / 2
        
        elif eps.shape[1] == 3:
            
            if configuration == '90':
                return np.max(eps[:,1]) - (np.max(eps[:,0]) + np.max(eps[:,2])) / 2
        
            elif configuration == '120':                
                return (np.max(eps[:,0]) - np.max(eps[:,2])) / np.sqrt(3)              
    
    def from_material(m):
        
        '''
        Obtain the thermoelastic coefficient choosing standard materials
        
        Input:
        -----
        m : {'steel', 'aluminium', 'titanium', 'epoxy', 'magnesium', 'glass'}
            Standard materials
        
        Return:
        ------
        km : float
            The thermoelastic coefficient [Pa^-1]
        
        '''
        
        material = {'steel' : 3.5 * 10**-12,
                    'aluminium' : 8.8 * 10**-12,
                    'titanium' : 3.5 * 10**-12,
                    'epoxy' : 6.2 * 10**-11,
                    'magnesium' : 1.4 * 10**-11,
                    'glass' : 3.85 * 10**-12}
        
        return material[m]
    
    def from_strain_gauge(data, fs, fl, E, ni, strain, location):
        
        '''
        Obtain the thermoelastic coefficient thorugh strain-gauge calibration procedure
        
        Input:
        -----
        data : array_like, [frames, height, width] 
            Sequence of thermal images [°C]
        
        fs : float
            Sampling frequency [Hz]
        
        fl : float
            Load frequency of harmonic excitation [Hz]
            
        E : float
            Young Modulus of the material [Pa]
        
        ni : float
            Poisson's ratio
            
        strain : float
            Strain measured using strain-gauges [e]
            
        location : int, (x,y,w,h)
            ROI coordinates of the area where the strain-gauges are bonded 
            
        Return:
        ------
        km : float
            The thermoelastic coefficient [Pa^-1]
        
        '''
        
        (x,y,w,h) = location
        mag, _ = pyLIA.LIA(data,fs,fl)
        mag_avg = np.mean(mag[y:(y+h),x:(x+w)])

        return (mag_avg * (1-ni)) / (E * strain)
    